<a href="https://colab.research.google.com/github/kkrree/todaysPython/blob/master/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 머신러닝 텍스트 분류실습
source : https://www.youtube.com/playlist?list=PLaTc2c6yEwmp35y9Z868VlV9NQpG-eBb6

머신러닝 기법 분류
![이미지](https://i.imgur.com/b6wcQ7f.png)

데이터 분석 및 머신러닝 적용 프로세스
![이미지](https://i.imgur.com/li0a2zj.png)



```
# 참고
scikit-learn 1.0 이상 버전 사용 권장 (sklearn__version__ 으로 확인)
1.0 이하의 버전이라면 conda prompt 혹은 터미널을 열어 아래 명령어로 업그레이드
conda : conda update scikit-learn
pip install scikit-learn --upgrade
```


### 데이터 셋 준비

In [ ]:
#드라이브에서 파일 가져오기
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Nexon_Assignment/"
content =pd.read_csv(path + "content.csv")
comment =pd.read_csv(path + "comment.csv")

In [ ]:
# 라이브러리 불러오기
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib 을 로드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### 텍스트 데이터 수치 형태로 변경

In [ ]:
# sklearn.feature_extraction.text의 CountVectorizer 로 BOW 벡터화
# fit_transform 으로 훈련 및 변환
from sklearn.feature_extraction.text import CountVectorizer 
cvect = CountVectorizer()
dtm = cvect.fit_transform(content['title']) 
dtm # sparse matrix 희소행렬로 변환된 것 확인

### DTM(document-term matrix)

In [ ]:
# 피처 가져오기
featureNames = cvect.get_feature_names_out()
featureNames

In [ ]:
# 단어 사전 보기
cvect.vocabulary_

In [ ]:
# dtm(document-term matrix)
pd.DataFrame(dtm.toarray(), columns = featureNames)

### 문제와 정답 만들기

In [ ]:
# 독랍변수로 사용할 x 변수에 dtm array를 할당
X = dtm.toarray()
X

In [ ]:
# 종속변수로 사용할 변수에 정답값인  값 할당
Y = content['viewCount']
Y

### 데이터셋 나누기
* 현재 예제에서는 임의로 나눔
* test_train_split 을 활용하여 나눌 수 있음

In [ ]:
# 8:2 의 비율로 구하기 위해 전체 데이터의 행에서 80% 위치에 해당되는 값을 구해서 split_count 라는 변수에 담습니다.
splitCount = int(content.shape[0]*0.8)
splitCount

In [ ]:
# 학습 세트 만들기
X_train = X[:splitCount]
Y_train = Y[:splitCount]
print(X_train.shape, Y_train.shape) # 행 갯수 같은지 확인
X_train, Y_train

In [ ]:
# 예측 세트 만들기
X_test = X[splitCount:]
Y_test = Y[splitCount:]
print(X_test.shape, Y_test.shape) # 행 갯수 같은지 확인
X_test, Y_test